# Bet Size from Computational Optimization
In the previous notebook "Optimal Betsizing for Simultaneous Identical Bets" it was shown how one can find the optimal bet size (K), and how it is equivilent to the Kelly optimal, by differentiating the geometric expected value equation of simultaneous bets. An alternative and computationally easier approach may be to just use a optimization algorithm to directly find the optimal bet that maximized the GEV equation. This notebook will attempt this alternative approach.

## The Toy Discrete Problem

The problem to be solved is to find the optimal bet size K and maximum GEV of N number of simultaneous and identical toy bets. The toy bets have two outcomes with payoffs W and L that are percentages of the bet size K.

The proceedure will work as follows:
- Get all ${2}^{N}$ combinations of outcomes
- Compute the joint probabilities and ending wealths for each outcome
- Construct GEV equation given the joint endingwealths
- Find the K that miximises the GEV using an optimization algorithm

The proceedure will be coded into a single function with parameters N, win_return, loss_return, win_probaility and outputs the K and max GEV. The sensitivities of the parameters can then be investigated.

The first step is to get all combinations of length N with two outcomes. This can be done by interation and a representing the combination as a tuple all of which contained in a list. The code can be seen below.

In [2]:
import itertools as it
# probabilities of outcomes are the win_prob and 1-win_prob
probabilties = [0.9, 0.1]
list_prob = list(it.product(probabilties, repeat=3))
print(list_prob)
# returns are [win_return, loss_return]
returns = [0.2, -1]
list_ret = list(it.product(returns, repeat=3))
print(list_ret)

[(0.9, 0.9, 0.9), (0.9, 0.9, 0.1), (0.9, 0.1, 0.9), (0.9, 0.1, 0.1), (0.1, 0.9, 0.9), (0.1, 0.9, 0.1), (0.1, 0.1, 0.9), (0.1, 0.1, 0.1)]
[(0.2, 0.2, 0.2), (0.2, 0.2, -1), (0.2, -1, 0.2), (0.2, -1, -1), (-1, 0.2, 0.2), (-1, 0.2, -1), (-1, -1, 0.2), (-1, -1, -1)]


The next steps are to compute the joint probabilities and returns for each combinations. The joint probsbilities are simply the product of the combinations probability. The return of the combination is just the average return of each bet.

In [3]:
P = []
for combo in list_prob:
    product = 1
    for ele in combo: 
        product *= ele
    P.append(product)
print(P)

[0.7290000000000001, 0.08100000000000002, 0.08100000000000002, 0.009000000000000001, 0.08100000000000002, 0.009000000000000001, 0.009000000000000003, 0.0010000000000000002]


In [4]:
r =[]
for combo in list_ret:
    r.append(sum(combo)/3)
print(r)

[0.20000000000000004, -0.19999999999999998, -0.20000000000000004, -0.6, -0.20000000000000004, -0.6, -0.6, -1.0]


Finally the joint returns and proabilities are to be represented as arrays and then put into the GEV equation that is to be maximized.

${GEV} = e^{ \sum {P}_{i}\ln{(1+{K}*{r}_{i})} }$

$P = [{P}_0, ... , {P}_{n}]$

$r = [{r}_0, ... , {r}_{n}]$

Where ${P}_{i}$ and ${r}_{i}$ are the probabilities and returns of the ${i}^{th}$ outcome combination of the N bets.

In [5]:
import numpy as np
P = np.array(P)
r = np.array(r)

In [6]:
def GEV(K,_prob,_ret):
    '''This function calculates the geometric expected value given the ending wealths of all outcomes
    :param _prob: An array of all the probabilities of each outcome
    :param _ret: An array of the return of each outcome
    :param K: The betsize as a percentage of total portfolio alocated to all bets
    :return: It returns the the GEV*-1. This is so a minimize algortithm can be used to find the optimal K
    '''
    summation = _prob*np.log(1+K*_ret)
    gev = np.exp(sum(summation))
    return -gev

In [11]:
# testing out the GEV function with known values. For the parameters used above and the optimal K of 0.9585 the GEV should equal 1.0511
GEV(0.9585,P,r)

-1.0511086867367478

### Optimizing the GEV

Next we will try to find the optimal K for the GEV equation using scipy optimaztion algorithms. 

In [9]:
from scipy.optimize import minimize
x0 = np.array([0.5])
result = minimize(GEV, x0, method='nelder-mead',
               args=(P, r), options={'xatol': 1e-8, 'disp': True})
result

Optimization terminated successfully.
         Current function value: -1.051109
         Iterations: 30
         Function evaluations: 61


C:\Users\luciu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


 final_simplex: (array([[0.95849557],
       [0.95849558]]), array([-1.05110869, -1.05110869]))
           fun: -1.0511086867436512
       message: 'Optimization terminated successfully.'
          nfev: 61
           nit: 30
        status: 0
       success: True
             x: array([0.95849557])

The simplex minimzation algorthim found the optimal K for the parameters used throughout this notebook as expected. We have shown that this is a viable computational method for finding K. Now all that is left is to turn put all the pieces together into a function that takes a varied parameters.

## Putting All the Pieces Together

In [12]:
import itertools as it
import numpy as np
from scipy.optimize import minimize

def getOptimalK(num_bets, win_return, loss_return, win_probability):
    '''
    This function finds the optimal bet size K for a number of identical and simultaneous bets.
    :param num_bets: The integer number of simultaneous bets
    :param win_return: The expected return of a single bet if it wins
    :param loss_return: The expected loss of a single bet if it loses
    :param win_probability: The single bet probability that a bet will win
    :return: [K, GEV] K is the optimal betsize of the combined bets... 
                      GEV is the geometric expected value of all the bets using bet size K
    '''
    # Prevent dimensionality explosion
    num_bets = min(num_bets, 20)
    
    # Get the list of outcome combinations of N bets
    combo_prob = list(it.product([win_probability, 1-win_probability], repeat=num_bets))
    combo_ret = list(it.product([win_return, loss_return], repeat=num_bets))
    
    # compute joint probabilties of outcomes
    P = []
    for combo in combo_prob:
        product = 1
        for i in combo: 
            product *= i
        P.append(product)
    P = np.array(P)
    
    #compute joint returns of outcomes
    r =[]
    for combo in combo_ret:
        r.append(sum(combo)/num_bets)
    r = np.array(r)
    
    # Find K that minimizes the GEV
    x0 = np.array([0.5])
    result = minimize(GEV, x0, method='nelder-mead', args=(P, r), options={'xatol': 1e-8, 'disp': True})
    
    return [result.final_simplex[0][0],-result.final_simplex[1][0]]  

Finally we have test the completed optimizing function using the same outcome parameters and looking at n = 2, 3 and 4.

In [13]:
print('Optimal results for 2 bets: ',getOptimalK(2,0.2,-1,0.9))

Optimization terminated successfully.
         Current function value: -1.035776
         Iterations: 28
         Function evaluations: 58
Optimal results for 2 bets:  [array([0.74164376]), 1.035776080602608]


In [14]:
print('Optimal results for 2 bets: ',getOptimalK(3,0.2,-1,0.9))

Optimization terminated successfully.
         Current function value: -1.051109
         Iterations: 30
         Function evaluations: 62
Optimal results for 2 bets:  [array([0.95849557]), 1.0511086867436512]


C:\Users\luciu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


In [15]:
print('Optimal results for 2 bets: ',getOptimalK(4,0.2,-1,0.9))

Optimization terminated successfully.
         Current function value: -1.060827
         Iterations: 33
         Function evaluations: 66
Optimal results for 2 bets:  [array([0.99724693]), 1.0608265345424128]


C:\Users\luciu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


In [19]:
print('Optimal results for 2 bets: ',getOptimalK(20,0.2,-1,0.9))

C:\Users\luciu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -1.076885
         Iterations: 51
         Function evaluations: 102
Optimal results for 2 bets:  [array([0.99999999]), 1.0768846720229204]
